In [1]:
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
from mudata import MuData
import mudata as md
from anndata import AnnData
import pyranges as pr
import bioframe as bf
import os

from RangeAnnData import *

In [2]:
os.chdir('/Users/qian/Desktop/2019-23_Lipka_Lab/EpiOncoRas/')

In [3]:
rna = pd.read_csv('rna_counts/H6c7_combined_num_reads.tsv', sep='\t')
counts = rna[rna.columns[8:]]
coords = rna[rna.columns[:8]]
coords.columns = ['chrom', 'start', 'end'] + list(coords.columns[3:])

adata = RangeAnnData(counts.transpose())
adata.obs_names = rna.columns[8:]
adata.var_names = rna['gene_id']
adata.set_coord(coords.set_index('gene_id'))

/Users/qian/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/79/cbc9h56s5v7gbsvc3kk0lwmw0000gq/T/ipykernel_62185/948008376.py:6: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = RangeAnnData(counts.transpose())
/Users/qian/opt/miniconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
epic = pd.read_feather('epic_array/h6c7_batch_corrected_by_channel.feather').set_index('sample_id')

epic_coords = pd.read_csv('../Annotation/infinium-methylationepic-v-1-0-b5-manifest-file.csv')
epic_coords = epic_coords.set_index('IlmnID')[['CHR_hg38', 'Start_hg38', 'End_hg38']].dropna()
epic_coords.columns = ['chrom', 'start', 'end']
epic_coords['start'] = epic_coords['start'].astype(int)
epic_coords['end'] = epic_coords['end'].astype(int)

/Users/qian/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
epic = epic.loc[np.intersect1d(epic.index, epic_coords.index)]
epic_coords = epic_coords.loc[np.intersect1d(epic.index, epic_coords.index)]

In [13]:
epic_data = RangeAnnData(epic.transpose())
epic_data.obs_names = rna.columns[8:]
epic_data.var_names = epic.index
epic_data.set_coord(epic_coords)

/var/folders/79/cbc9h56s5v7gbsvc3kk0lwmw0000gq/T/ipykernel_62185/2468243973.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  epic_data = RangeAnnData(epic.transpose())


In [57]:
mdata = RangeMuData({"rna": adata, "epic": epic_data})

In [21]:
bf.select(epic_coords, 'chr1:1-1000000')

,chrom,start,end
IlmnID,,,
cg00113194,chr1,949488,949490
cg00145911,chr1,932557,932559
cg00328058,chr1,965681,965683
cg00380574,chr1,983308,983310
cg00582671,chr1,936854,936856
...,...,...,...
cg27009105,chr1,925581,925583
cg27050229,chr1,921312,921314
cg27346986,chr1,870225,870227


In [27]:
tss_range_upstream = 1000
tss_range_downstream = 500
tss_range = coords.copy()
tss_range['end'] = tss_range['start'] + tss_range_downstream
tss_range['tss_start'] = tss_range['start'] - tss_range_upstream
tss_range['start'] = tss_range['tss_start']

In [33]:
tss_range['chrom'] = 'chr' + tss_range['chrom'].astype(str)

In [34]:
def groupby_agg(adata, granges):
    # coord = pr.PyRanges(adata.varm['coord'].reset_index())
    varm = adata.varm['coord'].copy()
    varm['idx'] = varm.index
    gb_gr_idx = bf.overlap(varm, granges, how='inner').groupby(['chrom_', 'start_', 'end_'])['idx'].apply(list).dropna()
    lst = []
    for i in gb_gr_idx:
        lst.append(adata[:, i].X.mean())
    return pd.Series(lst, index=gb_gr_idx.index)

In [49]:
mean_epic_E37G = groupby_agg(epic_data[epic_data.obs_names[:5],], tss_range)

In [50]:
mean_epic_EV = groupby_agg(epic_data[epic_data.obs_names[5:10],], tss_range)

In [65]:
mean_methylation = pd.DataFrame([mean_epic_E37G, mean_epic_EV], index=['E37G', 'EV']).transpose()
mean_methylation.index.set_names(['chrom', 'start', 'end'], inplace=True)
mean_methylation = mean_methylation.join(tss_range.set_index(['chrom', 'start', 'end'])).set_index('gene_id')

In [70]:
mean_methylation[(mean_methylation['E37G'] - mean_methylation['EV']).abs() >= 0.05]

,E37G,EV,score,strand,name,exonic_length,tss_start
gene_id,,,,,,,
ENSG00000204481.6,0.5790182,0.6409272,.,-,PRAMEF14,2523,13667269
ENSG00000222624.1,0.5826139,0.5301477,.,+,RNU2-15P,191,65880499
ENSG00000250833.1,0.72515804,0.67043173,.,+,DNM1P17,426,30641635
ENSG00000223462.2,0.20645413,0.26329732,.,+,RP11-285G1.9,425,45431956
ENSG00000235376.5,0.516387,0.46560168,.,+,RP11-332O19.5,2130,105004644
ENSG00000254695.1,0.6557648,0.60413355,.,+,RP11-396O20.1,471,15664431
ENSG00000255375.1,0.4806428,0.41132194,.,+,RP1-65P5.5,646,32153245
ENSG00000135387.15,0.5615539,0.62429726,.,+,CAPRIN1,7689,34072230
ENSG00000221586.1,0.68650854,0.738568,.,-,MIR1261,82,90601289
